In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
#from IPython.display import clear_output
#!pip3 install -U lazypredict
#!pip3 install -U pandas #Upgrading pandas

In [3]:
'''
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots



from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split


from lightgbm import LGBMClassifier
import lazypredict
from lazypredict.Supervised import LazyClassifier

import time
import warnings
warnings.filterwarnings('ignore')
'''

"\nimport numpy as np\nimport pandas as pd\nimport seaborn as sns\nimport plotly.express as px\nimport matplotlib.pyplot as plt\nimport plotly.graph_objects as go\nfrom plotly.subplots import make_subplots\n\n\n\nfrom sklearn.impute import SimpleImputer\nfrom sklearn.metrics import accuracy_score\nfrom sklearn.preprocessing import LabelEncoder\nfrom sklearn.model_selection import StratifiedKFold, train_test_split\n\n\nfrom lightgbm import LGBMClassifier\nimport lazypredict\nfrom lazypredict.Supervised import LazyClassifier\n\nimport time\nimport warnings\nwarnings.filterwarnings('ignore')\n"

# 라이브러리 및 데이터 불러오기
- 라이브러리 버전 확인 필수

# 2. 탐색적 자료 분석 (EDA) 
- 데이터 시각화, 산점도, 막대 그래프 등, 그래프 해석해서 설명을 달아야 한다. 약간의 데이터 전처리

# 3. 데이터 전처리 - ml모형 돌리기 위해 표준화,  원 핫 인코딩, -> feature engineering
- 파생변수(도출변수) 왜?? 이 변수를 만들었는지에 대한 설명 필요

# 4. 머신러닝 모형 개발
- 로지스틱, 모형 등 (모형에 대한 설명이 들어가 있어야 한다.)
- 모형 여러개 말구 한 두개 정도만

# 5. 모형 펴가
- 훈련 데이터 쪼개 훈련 데이터와 검증 데이터를 가지고
- 테스트 데이터 건드는 거 아님
- 정확도 비교 
- (혼동행렬 Confusion Matrix) 설명

# 6. 제출
- 양식 : 추후에 샘플 주심.

# Regerence 
- 참고한거 있으면 참고한 페이지 추가
- 저자 이름, 글제목, 링크 주소


In [4]:
STRATEGY = 'median'
FOLDS = 5
RANDOM_STATE = 12

In [5]:
train = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Transported 예측

In [6]:
test = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


결측치가 있어 보임

In [8]:
train.isna().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [9]:
# train['Cunsumption'] = train['RoomService'] + train['ShoppingMall'] + train['Spa'] + train['VRDeck']

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


나이대별로 묶고 전송했는지 안했는지

In [11]:
train['Age'].describe()

count    8514.000000
mean       28.827930
std        14.489021
min         0.000000
25%        19.000000
50%        27.000000
75%        38.000000
max        79.000000
Name: Age, dtype: float64

In [12]:
train['HomePlanet'].value_counts()

Earth     4602
Europa    2131
Mars      1759
Name: HomePlanet, dtype: int64

출발지별 보기 시각화

In [13]:
import plotly.graph_objects as go
fig = go.Figure()
# fig.add_trace(go.bar(x = train['HomePlanet'], y = train['Transported']))
fig.show()

In [14]:
train.sort_values(['Age'])

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
1763,1874_01,Earth,True,G/305/P,TRAPPIST-1e,0.0,False,0.0,0.0,0.0,0.0,0.0,Armene Woodgezalez,False
774,0811_03,Mars,False,F/152/S,TRAPPIST-1e,0.0,False,0.0,0.0,0.0,0.0,0.0,Phams Nutte,True
773,0811_02,Mars,True,F/152/S,TRAPPIST-1e,0.0,False,0.0,0.0,0.0,0.0,0.0,Pan Nutte,True
5982,6334_04,Earth,True,G/1028/S,PSO J318.5-22,0.0,False,0.0,0.0,0.0,0.0,0.0,Vick Jenson,True
771,0809_04,Mars,False,F/151/S,TRAPPIST-1e,0.0,False,0.0,0.0,0.0,0.0,0.0,Wales Meake,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8274,8835_01,Earth,True,G/1425/S,TRAPPIST-1e,NaN,False,0.0,0.0,0.0,0.0,0.0,Shalle Bartines,False
8301,8862_03,Europa,True,C/329/S,TRAPPIST-1e,NaN,False,0.0,0.0,0.0,0.0,0.0,Alchib Myling,True
8374,8956_04,Earth,False,G/1453/P,TRAPPIST-1e,NaN,False,194.0,1.0,10.0,629.0,0.0,Krisa Bonnondry,False
8407,8988_01,Earth,True,G/1448/S,TRAPPIST-1e,NaN,False,0.0,0.0,0.0,0.0,0.0,Maen Fowlesterez,True


최소 나이가 0살????

최대 나이가 79살

-> 0 - 

nan은 어쩌지

# 흐름 잡기

In [15]:
train.iloc[:,:].describe().T.sort_values(by = 'std', ascending = False)\
    .style.background_gradient(cmap="GnBu")\
    .bar(subset = ['max'], color = '#BB0000')\
    .bar(subset = ['mean'], color = 'green')

,count,mean,std,min,25%,50%,75%,max
FoodCourt,8510.000000,458.077203,1611.489240,0.000000,0.000000,0.000000,76.000000,29813.000000
VRDeck,8505.000000,304.854791,1145.717189,0.000000,0.000000,0.000000,46.000000,24133.000000
Spa,8510.000000,311.138778,1136.705535,0.000000,0.000000,0.000000,59.000000,22408.000000
RoomService,8512.000000,224.687617,666.717663,0.000000,0.000000,0.000000,47.000000,14327.000000
ShoppingMall,8485.000000,173.729169,604.696458,0.000000,0.000000,0.000000,27.000000,23492.000000
Age,8514.000000,28.827930,14.489021,0.000000,19.000000,27.000000,38.000000,79.000000


- 푸드코트에서 사용량이 제일 많고 VR이랑 spa가 비등비등 그다음이 룸서비스랑 쇼핑몰

* 행의 결측값 최대값은 3이고 가장 낮은 값은 결측값이 없습니다.
* 흥미롭게도, 결측값 분포 (행 기준)는 열차와 테스트 데이터 집합 간에 거의 동일합니다.
* 결측값이 없는 관측치(행 기준)의 약 76%가 있습니다.
* 결측값이 1 ~ 3개인 관측치의 나머지 24%(행 기준)

* 12가지 특징 중 6가지 특징이 연속적이며, 2가지 특징이 텍스트 데이터이며, 4가지 특징이 범주형입니다.
* HomePlanet 및 Destination에는 3개의 다른 고유 값이 있습니다.
* CryoSleep과 VIP는 부울 기능

In [16]:
'''
text_features = ["Cabin", "Name"]
cat_features = [col for col in train.columns if df[col].nunique() < 25 and col not in text_features ]
cont_features = [col for col in train.columns if df[col].nunique() >= 25 and col not in text_features ]

del df
print(f'\033[94mTotal number of features: {len(FEATURES)}')
print(f'\033[94mNumber of categorical features: {len(cat_features)}')
print(f'\033[94mNumber of continuos features: {len(cont_features)}')
print(f'\033[94mNumber of text features: {len(text_features)}')

labels=['Categorical', 'Continuos', "Text"]
values= [len(cat_features), len(cont_features), len(text_features)]
colors = ['#DE3163', '#58D68D']

fig = go.Figure(data=[go.Pie(
    labels=labels, 
    values=values, pull=[0.1, 0, 0 ],
    marker=dict(colors=colors, 
                line=dict(color='#000000', 
                          width=2))
)])
fig.show()
'''

'\ntext_features = ["Cabin", "Name"]\ncat_features = [col for col in train.columns if df[col].nunique() < 25 and col not in text_features ]\ncont_features = [col for col in train.columns if df[col].nunique() >= 25 and col not in text_features ]\n\ndel df\nprint(f\'\x1bTotal number of features: {len(FEATURES)}\')\nprint(f\'\x1bNumber of categorical features: {len(cat_features)}\')\nprint(f\'\x1bNumber of continuos features: {len(cont_features)}\')\nprint(f\'\x1bNumber of text features: {len(text_features)}\')\n\nlabels=[\'Categorical\', \'Continuos\', "Text"]\nvalues= [len(cat_features), len(cont_features), len(text_features)]\ncolors = [\'#DE3163\', \'#58D68D\']\n\nfig = go.Figure(data=[go.Pie(\n    labels=labels, \n    values=values, pull=[0.1, 0, 0 ],\n    marker=dict(colors=colors, \n                line=dict(color=\'#000000\', \n                          width=2))\n)])\nfig.show()\n'

In [17]:
'''train_age = train.copy()
test_age = test.copy()
train_age['type'] = 'Train'
test_age['type'] = 'Test'

ageDf = pd.concat([train_age,test_age])
fig = px.histogram(data_frame = ageDf,
                  x = 'Age',
                  color = 'type',
                  #color_discreate_sequence = ['#58D68D','#DE3163'],
                  marginal='box',
                  nbins = 100
                  )
fig.update_layout(title = 'Deistribution of Afe', title_x = 0.5)
fig.show()
'''

"train_age = train.copy()\ntest_age = test.copy()\ntrain_age['type'] = 'Train'\ntest_age['type'] = 'Test'\n\nageDf = pd.concat([train_age,test_age])\nfig = px.histogram(data_frame = ageDf,\n                  x = 'Age',\n                  color = 'type',\n                  #color_discreate_sequence = ['#58D68D','#DE3163'],\n                  marginal='box',\n                  nbins = 100\n                  )\nfig.update_layout(title = 'Deistribution of Afe', title_x = 0.5)\nfig.show()\n"

In [18]:
#fig = px.imshow(train.corr(), text_auto= True, aspect='auto', color_continuous_scale = 'viridis')
#fig.show()

-데이터 처리

simpleimputer = fillna 메서드 역할

In [19]:
'''# 결측치의 귀속
imputer_cols = ['Age','FoodCourt','ShoppingMall','Spa','VRDeck','RoomService']
# 결측치 중간값으로 채우겠다.
imputer = SimpleImputer(strategy=STRATEGY )
# 결측치 중간값으로 채우는 훈련 학습
imputer.fit(train[imputer_cols])
# 데이터 변환
train[imputer_cols] = imputer.transform(train[imputer_cols])
test[imputer_cols] = imputer.transform(test[imputer_cols])
# 텍스트 컬럼인 Homeplanet는 z로 채운다.
train['HomePlanet'].fillna('Z',inplace=True)
test['HomePlanet'].fillna('Z',inplace =True)
'''

"# 결측치의 귀속\nimputer_cols = ['Age','FoodCourt','ShoppingMall','Spa','VRDeck','RoomService']\n# 결측치 중간값으로 채우겠다.\nimputer = SimpleImputer(strategy=STRATEGY )\n# 결측치 중간값으로 채우는 훈련 학습\nimputer.fit(train[imputer_cols])\n# 데이터 변환\ntrain[imputer_cols] = imputer.transform(train[imputer_cols])\ntest[imputer_cols] = imputer.transform(test[imputer_cols])\n# 텍스트 컬럼인 Homeplanet는 z로 채운다.\ntrain['HomePlanet'].fillna('Z',inplace=True)\ntest['HomePlanet'].fillna('Z',inplace =True)\n"

In [20]:
'''
label_cols=["HomePlanet", "CryoSleep","Cabin", "Destination" ,"VIP"]
def label_encoder(train,test,columns):
    for col in columns:
        train[col] = train[col].astype(str)
        test[col] = test[col].astype(str)
        train[col] = LabelEncoder().fit_transform(train[col])
        test[col] = LabelEncoder().fit_transform(test[col])
    return train, test
train, test = label_encoder(train,test,label_cols)
'''

'\nlabel_cols=["HomePlanet", "CryoSleep","Cabin", "Destination" ,"VIP"]\ndef label_encoder(train,test,columns):\n    for col in columns:\n        train[col] = train[col].astype(str)\n        test[col] = test[col].astype(str)\n        train[col] = LabelEncoder().fit_transform(train[col])\n        test[col] = LabelEncoder().fit_transform(test[col])\n    return train, test\ntrain, test = label_encoder(train,test,label_cols)\n'

- 다 문자열로 바꿔주고 